This script is a sandbox for feature engineering.  
A function is later written in the script, "fe_functions.ipynb" that utilizes this code.  


In [2]:
import pandas as pd
import numpy as np
import pickle
import sqlite3

In [11]:
#connect to the MIMIC DEMO data. 
con = sqlite3.connect('MIMIC3_full.db')

In [15]:
#admissions table
admitdf = pd.read_sql('select * from admission', con)

In [16]:
#icu table
icudf =  pd.read_sql('select * from icu', con)

In [18]:
#procedures table
procdf = pd.read_sql('select * from procedure', con)

In [19]:
#prescriptions table
drugdf = pd.read_sql('select * from prescription', con)

In [20]:
#close the connection
con.close()

In [24]:
#lower case the columns
admitdf.columns = admitdf.columns.str.lower()

In [27]:
icudf.columns = icudf.columns.str.lower()

In [31]:
procdf.columns = procdf.columns.str.lower()
drugdf.columns = drugdf.columns.str.lower()

In [26]:
#ensure that we are selecting only our columns of interest.
fields = ["subject_id","hadm_id","deathtime", "admission_type", "admission_location", 
          "discharge_location", "insurance", "diagnosis", "hospital_expire_flag"]
admitdf = admitdf[fields]

In [29]:
fields =["subject_id", "hadm_id", "icustay_id", "first_careunit", "last_careunit",
        "first_wardid", "last_wardid", "los"]
icudf = icudf[fields]

## Identify those who died in the ICU

In [30]:
#first, identify those invididuals who died during their hospital/ICU stay
deaths = admitdf[admitdf['discharge_location'].isin(['DEAD/EXPIRED'])]
#merge the outcome onto the ICU base dataframe via a left merge
icu_admin = pd.merge(icudf, deaths, how='left', on='hadm_id')
icu_admin['discharge_location'].value_counts()
#make the df look nice
icu_full = (
            icu_admin.
            drop(columns=['subject_id_y']).
            rename(columns={"subject_id_x": "subject_id"})
)
icu_full

,subject_id,hadm_id,icustay_id,first_careunit,last_careunit,first_wardid,last_wardid,los,deathtime,admission_type,admission_location,discharge_location,insurance,diagnosis,hospital_expire_flag
0,268,110404,280836,MICU,MICU,52,52,3.2490,2198-02-18 03:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,DYSPNEA,1.0
1,269,106296,206613,MICU,MICU,52,52,3.2788,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,270,188028,220345,CCU,CCU,57,57,2.8939,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,271,173727,249196,MICU,SICU,52,23,2.0600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,272,164716,210407,CCU,CCU,57,57,1.6202,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,94944,143774,201233,CSRU,CSRU,15,15,2.1894,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61528,94950,123750,283653,CCU,CCU,7,7,2.4942,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61529,94953,196881,241585,SICU,SICU,57,57,0.9259,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61530,94954,118475,202802,CSRU,CSRU,15,15,2.3346,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Procedures

In [32]:
#procdf = pd.read_csv("https://physionet.org/files/mimiciii-demo/1.4/PROCEDUREEVENTS_MV.csv?download")
#take the top eight procedures from the demo data 
topprocs = procdf['ordercategoryname'].value_counts().head(8).index.tolist()
#filter the data to only those from the top eight procedures
procdf = procdf[procdf['ordercategoryname'].isin(topprocs)].reset_index()
#remove NAs
procdf = procdf[procdf['icustay_id'].notna()]
#create a count dataframe that will later be merged
myproc_counts = procdf.groupby(['subject_id', 'icustay_id', 'ordercategoryname']).size().reset_index(name='counts')
myproc_counts_long = myproc_counts.pivot(index = ['subject_id','icustay_id'], 
                                         columns = 'ordercategoryname',
                                         values = 'counts').reset_index()
#repalce the NAs with 0s
myproc_counts_long = myproc_counts_long.replace(np.nan,0)
#myproc_counts_long = myproc_counts_long.reset_index(drop=True, inplace=True)
myproc_counts_long = myproc_counts_long.rename_axis(None).drop(columns=['subject_id'])
#place the columns into a list for later use
procedure_columns = ['Dialysis', 'Imaging', 'Intubation/Extubation',
       'Invasive Lines', 'Peripheral Lines', 'Procedures',
       'Significant Events', 'Ventilation']
#pickle the data in a crosstable for later uses
with open("../../crosstables/procedure_list.txt", "wb") as fp:   #Pickling
    pickle.dump(procedure_columns, fp)
myproc_counts_long.head()

ordercategoryname,icustay_id,Communication,Imaging,Intubation/Extubation,Invasive Lines,Peripheral Lines,Procedures,Significant Events,Ventilation
0,234044.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
1,290505.0,0.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0
2,241249.0,0.0,3.0,1.0,0.0,5.0,3.0,0.0,1.0
3,291697.0,0.0,1.0,0.0,0.0,2.0,3.0,0.0,0.0
4,252542.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0


## Drugs

In [33]:
#drugdf = pd.read_csv("https://physionet.org/files/mimiciii-demo/1.4/PRESCRIPTIONS.csv?download")
top_twenty = drugdf['formulary_drug_cd'].value_counts().head(20).index.tolist()
#filter rows to only these drugs
drugdf = drugdf[drugdf['formulary_drug_cd'].isin(top_twenty)].reset_index()
drugdf = drugdf[drugdf['icustay_id'].notna()]
#now start making counts based on person and icu_stay_id
mycounts = drugdf.groupby(['subject_id', 'icustay_id', 'formulary_drug_cd']).size().reset_index(name='counts')
mycounts_long = mycounts.pivot(index = ['subject_id','icustay_id'], 
                               columns = 'formulary_drug_cd', values = 'counts').reset_index()
mycounts_long = mycounts_long.replace(np.nan,0).drop(columns=['subject_id'])
prescription_list = ['ACET325', 'CALG1I', 'D5W1000', 'D5W250', 'FURO40I',
       'HEPA5I', 'INSULIN', 'KCL20P', 'KCL20PM', 'KCLBASE2', 'LR1000',
       'MAG2PM', 'METO25', 'MORP2I', 'NACLFLUSH', 'NS1000', 'NS250', 'NS500',
       'VANC1F', 'VANCOBASE']
#pickle the data for later uses into the crosstables folder
with open("../../crosstables/prescription_list.txt", "wb") as dl:   #Pickling
    pickle.dump(prescription_list, dl)
mycounts_long.head()

formulary_drug_cd,icustay_id,ACET325,D5W250,DOCU100,FURO40I,HEPA5I,INSULIN,KCL20PM,KCLBASE2,LR1000,...,METO25,METO5I,MICROK10,NACLFLUSH,NS100,NS1000,NS250,NS500,VANC1F,VANCOBASE
0,294638.0,1.0,2.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,228232.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0
2,220597.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,229441.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,232669.0,0.0,1.0,0.0,0.0,3.0,3.0,4.0,4.0,0.0,...,0.0,5.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0


Merge procedures and drugs to ICU data

In [34]:
#merge the counts data for final analysis
df1 = pd.merge(icu_full, mycounts_long, how='left', on='icustay_id')
#merge again for procedures to the final analytic dataset 
analyticdf = pd.merge(df1, myproc_counts_long, how='left', on='icustay_id')
analyticdf

,subject_id,hadm_id,icustay_id,first_careunit,last_careunit,first_wardid,last_wardid,los,deathtime,admission_type,...,VANC1F,VANCOBASE,Communication,Imaging,Intubation/Extubation,Invasive Lines,Peripheral Lines,Procedures,Significant Events,Ventilation
0,268,110404,280836,MICU,MICU,52,52,3.2490,2198-02-18 03:55:00,EMERGENCY,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,269,106296,206613,MICU,MICU,52,52,3.2788,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,270,188028,220345,CCU,CCU,57,57,2.8939,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,271,173727,249196,MICU,SICU,52,23,2.0600,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,272,164716,210407,CCU,CCU,57,57,1.6202,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,94944,143774,201233,CSRU,CSRU,15,15,2.1894,NaN,NaN,...,1.0,1.0,0.0,2.0,1.0,3.0,5.0,4.0,1.0,1.0
61528,94950,123750,283653,CCU,CCU,7,7,2.4942,NaN,NaN,...,1.0,1.0,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0
61529,94953,196881,241585,SICU,SICU,57,57,0.9259,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0
61530,94954,118475,202802,CSRU,CSRU,15,15,2.3346,NaN,NaN,...,0.0,0.0,0.0,3.0,1.0,3.0,4.0,5.0,1.0,1.0
